In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D,
    Flatten, Dense, Dropout, TimeDistributed, GlobalMaxPooling1D,
    GlobalMaxPooling2D, concatenate, BatchNormalization
    
)
from tensorflow.keras.optimizers import Adam

# For reproducibility
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)


2025-07-03 21:43:59.458241: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-03 21:43:59.467461: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751561039.478856   20688 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751561039.482354   20688 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1751561039.490705   20688 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Load your dataset
df = pd.read_csv("train.csv")

# Identify IMU and TOF columns
imu_cols = [col for col in df.columns if col.startswith(("acc_", "rot_", "thm_"))]
tof_cols = [col for col in df.columns if col.startswith("tof_")]
tof_sensors = ['tof_1', 'tof_2', 'tof_3', 'tof_4', 'tof_5']
tof_ordered = [f"{s}_v{v}" for s in tof_sensors for v in [0, 7, 56, 63]]

# Filter invalid rows
sensor_cols = imu_cols + tof_cols
df = df[~(df[sensor_cols] == -1).all(axis=1)]
df.replace(-1, np.nan, inplace=True)

# Group by sequence
sequences = df['sequence_id'].unique()
sequences.sort()

X_imu = []
X_tof = []
y_binary = []
y_gesture = []

for seq_id in sequences:
    seq_df = df[df['sequence_id'] == seq_id]

    # Skip if TOF or IMU missing
    if any(col not in seq_df.columns for col in imu_cols + tof_ordered):
        continue

    # Infer time_steps from this sequence
    time_steps = len(seq_df)

    imu_data = seq_df[imu_cols].fillna(0).values
    tof_data = seq_df[tof_ordered].fillna(0).values.reshape((time_steps, 5, 4))

    first_row = seq_df.iloc[0]
    if first_row['behavior'] != 'Performs gesture':
        continue

    X_imu.append(imu_data)
    X_tof.append(tof_data)
    y_binary.append(1 if first_row['sequence_type'] == 'Target' else 0)
    y_gesture.append(first_row['gesture'])

# Convert lists to arrays
X_imu_final = np.array(X_imu)
X_tof_final = np.array(X_tof)
y_binary_final = np.array(y_binary)
y_gesture_final = LabelEncoder().fit_transform(y_gesture)


In [ ]:
# ----------------------------------
# Step 3: Train-Validation Split
# ----------------------------------
X_imu_train, X_imu_val, X_tof_train, X_tof_val, yb_train, yb_val, yg_train, yg_val = train_test_split(
    X_imu_final, X_tof_final, y_binary_final, y_gesture_final,
    stratify=y_gesture_final, test_size=0.2, random_state=42
)

# ----------------------------------
# Step 4: Model Architecture
# ----------------------------------
def build_model(time_steps, imu_features, tof_shape, gesture_classes):
    # IMU 1D CNN
    imu_input = Input(shape=(time_steps, imu_features), name='imu_input')
    x1 = Conv1D(64, kernel_size=5, activation='relu', padding='same')(imu_input)
    x1 = MaxPooling1D(pool_size=2)(x1)
    x1 = Dropout(0.3)(x1)
    x1 = Conv1D(128, kernel_size=3, activation='relu', padding='same')(x1)
    x1 = GlobalAveragePooling1D()(x1)

    # TOF 2D CNN (with TimeDistributed)
    tof_input = Input(shape=tof_shape, name='tof_input')  # shape = (time_steps, 5, 4)
    x2 = TimeDistributed(Conv2D(32, (3, 3), padding='same', activation='relu'))(tof_input)
    x2 = TimeDistributed(MaxPooling2D(pool_size=(2, 2)))(x2)
    x2 = TimeDistributed(Dropout(0.3))(x2)
    x2 = TimeDistributed(Flatten())(x2)
    x2 = GlobalAveragePooling1D()(x2)

    # Merge IMU and TOF paths
    x = concatenate([x1, x2])
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.4)(x)

    # Outputs
    binary_output = Dense(1, activation='sigmoid', name='binary_output')(x)
    gesture_output = Dense(gesture_classes, activation='softmax', name='gesture_output')(x)

    model = Model(inputs=[imu_input, tof_input], outputs=[binary_output, gesture_output])
    return model

gesture_classes = len(np.unique(y_gesture_final))
model = build_model(
    time_steps=X_imu_final.shape[1],
    imu_features=X_imu_final.shape[2],
    tof_shape=X_tof_final.shape[1:],
    gesture_classes=gesture_classes
)
